In [1]:
!pip install beautifulsoup4

     |████████████████████████████████| 102kB 7.2MB/s ta 0:00:011


In [7]:
import urllib
from bs4 import BeautifulSoup
import pandas as pd

html = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").read()
soup = BeautifulSoup(html, 'html.parser')

In [8]:
table = soup.find('table', {'class': 'wikitable sortable'})

columns = [th.text.replace('\n', '') for th in table.find('tr').find_all('th')]
columns

trs = table.find_all('tr')[1:]
rows = list()
for tr in trs:
    rows.append([td.text.replace('\n', '').replace('\xa0', '') for td in tr.find_all('td')])
rows[:5]

df = pd.DataFrame(data=rows, columns=columns)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [9]:

df = df.loc[df['Borough']!='Not assigned']


df = df.groupby('Postcode').agg({'Borough':'first', 
                             'Neighbourhood': ', '.join}).reset_index()

df.loc[df['Neighbourhood']=='Not assigned','Neighbourhood']=df['Borough']

df.shape

(103, 3)


I use beatuiful soup to extract table from Wiki. After that i use df.loc and groupby to restructure the dataframe

In [10]:
file_name='http://cocl.us/Geospatial_data'
df2=pd.read_csv(file_name)

In [12]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
df = pd.merge(df,df2, how='left', left_on=['MKT_PLAN'], right_on=['Plan Code'], validate="many_to_one")   )

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
